In [1]:
import os
from configparser import SafeConfigParser

In [4]:
parser = SafeConfigParser()
parser.read("./seq2seq.ini",encoding="utf-8")

c:\Users\Administrator\Envs\py3.6.5_tf2\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.
  """Entry point for launching an IPython kernel.


['./seq2seq.ini']

In [5]:
_conf_ints = [ (key, int(value)) for key,value in parser.items('ints')]
_conf_floats = [ (key, float(value)) for key,value in parser.items('floats') ]
_conf_strings = [ (key, str(value)) for key,value in parser.items('strings') ]

In [6]:
dict(_conf_ints+_conf_floats+_conf_strings)

{'vocab_inp_size': 20000,
 'vocab_tar_size': 20000,
 'embedding_dim': 128,
 'train_epoch': 10,
 'layer_size': 512,
 'batch_size': 64,
 'max_length': 20,
 'number_work': 2,
 'min_loss': 0.2,
 'mode': 'train',
 'train_data': 'datasets',
 'seq_data': 'datasets/seq.data',
 'vocab_inp_path': 'datasets/inp.vocab',
 'vocab_tar_path': 'datasets/tar.vocab',
 'resource_data': 'datasets/xiaohuangji50w_nofenci.conv',
 'split_train_data': 'datasets/seq_data_',
 'e': 'E',
 'm': 'M',
 'model_data': 'model_data',
 'log_dir': 'log_dir'}

In [10]:
import json
import os
import re
import jieba
from zhon.hanzi import punctuation
from get_config import get_config
import io
import tensorflow as tf

In [21]:
conf_file = os.getcwd() + "/seq2seq.ini"
print(conf_file)
parser = SafeConfigParser()
parser.read(conf_file,encoding="utf-8")
_conf_ints = [(key, int(value)) for key, value in parser.items('ints')]
_conf_floats = [(key, float(value)) for key, value in parser.items('floats')]
_conf_strings = [(key, str(value)) for key, value in parser.items('strings')]
gConf= dict(_conf_ints+_conf_floats+_conf_strings)

conv_path = gConf["resource_data"]
vocab_inp_path = gConf["vocab_inp_path"]
vocab_tar_path = gConf["vocab_tar_path"]
vocab_inp_size = gConf["vocab_inp_size"]
vocab_tar_size = gConf["vocab_tar_size"]
seq_train = gConf["seq_data"]

e:\GitHub\ML\nlp\聊天机器人\tf2.6_prj/seq2seq.ini


c:\Users\Administrator\Envs\py3.6.5_tf2\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
def predata_util():
    # 判断训语料文件是否存在，如果不存在则提醒
    if not os.path.exists(conv_path):
        print("找不到conv文件")
        exit()

    # 新建一个文件，用于存放处理后的对话语料
    seq_train_file = open(seq_train, "w",encoding="utf-8")
    # 打开要处理的语料，逐条读取并进行数据处理
    with open(conv_path, encoding="utf-8") as f:
        one_conv = ""  # 存储一次完整的对话
        i = 0
        # 开始循环语料
        for line in f:
            line = line.strip("\n")
            line = re.sub(r"[%s]+" % punctuation, "", line)  # 去除标点符号
            if line == "":
                continue
            # 判断是否为一段对话的开始，如果是，则把刚处理过的语料保存下来
            if line[0] == gConf["e"]:
                if one_conv:
                    seq_train_file.write(one_conv[:-1] + "\n")
                    i = i + 1
                    if i % 1000 == 0:
                        print("处理进度：", i)
                one_conv = ""

            # 判断是否正在处理对华语剧，如果是则进行语料的拼接处理， 以及分词
            elif line[0] == gConf['m']:
                one_conv = one_conv + str(" ".join(jieba.cut(line.split(" ")[1]))) + "\t"  # 存储一次问答

    # 处理完成，关闭文件
    seq_train_file.close()

In [23]:
def create_vocab(lang, vocab_path, vocab_size):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token=3)
    tokenizer.fit_on_texts(lang)
    vocab = json.loads(tokenizer.to_json(ensure_ascii=False))
    vocab["index_word"] = tokenizer.index_word
    vocab["word_index"] = tokenizer.word_index
    vocab["document_count"] = tokenizer.document_count
    vocab = json.dumps(vocab,ensure_ascii=False)
    with open(vocab_path,'w',encoding="utf-8") as f:
        f.write(vocab)
    f.close()
    print("字典存在:{}".format(vocab_path))

In [24]:
def preprocess_sentence(w):
    w = 'start'+w +'end'
    return w

In [28]:
predata_util()

lines = io.open(seq_train,encoding="utf-8").readlines()
word_pairs = [[preprocess_sentence(w) for w in l.split("\t")] for l in lines]
input_lang,target_lang = zip(*word_pairs)

create_vocab(input_lang,vocab_inp_path,vocab_inp_size)
create_vocab(target_lang,vocab_tar_path,vocab_tar_size)

处理进度： 1000
处理进度： 2000
处理进度： 3000
处理进度： 4000
处理进度： 5000
处理进度： 6000
处理进度： 7000
处理进度： 8000
处理进度： 9000
处理进度： 10000
处理进度： 11000
处理进度： 12000
处理进度： 13000
处理进度： 14000
处理进度： 15000
处理进度： 16000
处理进度： 17000
处理进度： 18000
处理进度： 19000
处理进度： 20000
处理进度： 21000
处理进度： 22000
处理进度： 23000
处理进度： 24000
处理进度： 25000
处理进度： 26000
处理进度： 27000
处理进度： 28000
处理进度： 29000
处理进度： 30000
处理进度： 31000
处理进度： 32000
处理进度： 33000
处理进度： 34000
处理进度： 35000
处理进度： 36000
处理进度： 37000
处理进度： 38000
处理进度： 39000
处理进度： 40000
处理进度： 41000
处理进度： 42000
处理进度： 43000
处理进度： 44000
处理进度： 45000
处理进度： 46000
处理进度： 47000
处理进度： 48000
处理进度： 49000
处理进度： 50000
处理进度： 51000
处理进度： 52000
处理进度： 53000
处理进度： 54000
处理进度： 55000
处理进度： 56000
处理进度： 57000
处理进度： 58000
处理进度： 59000
处理进度： 60000
处理进度： 61000
处理进度： 62000
处理进度： 63000
处理进度： 64000
处理进度： 65000
处理进度： 66000
处理进度： 67000
处理进度： 68000
处理进度： 69000
处理进度： 70000
处理进度： 71000
处理进度： 72000
处理进度： 73000
处理进度： 74000
处理进度： 75000
处理进度： 76000
处理进度： 77000
处理进度： 78000
处理进度： 79000
处理进度： 80000
处理进度： 81000
处理进度： 82000
处理进度： 83000
处理进度： 84000
处

KeyboardInterrupt: 